<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/functionalcpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ
1. C++で関数型プログラミング風に書きたい、と思った。
1. functional programming in C++ という本のchapter01がネットで読めるので、導入だけでもいいと思って読んでいるところ。
1. https://itbook.store/files/9781617293818/chapter1.pdf

# 関数型プログラミング入門

In [ ]:
# 最初にでてくる、関数型でない書き方の例を、とりあえず動かす。
%%writefile countlines01.cpp
#include <iostream>
#include <fstream>
#include <vector>
std::vector<int> count_lines_in_files(const std::vector<std::string>& files) {
    std::vector<int> results;
    char c = 0;
    for (const auto& file : files) {
        int line_count = 0;
        std::ifstream in(file);
        while (in.get(c)) {
            if (c == '\n') {
                line_count++;
            }
        }
        results.push_back(line_count);
    }
    return results;
}
int main() {
    std::vector<std::string> files{"countlines01.cpp"};
    std::vector<int> results;

    results = count_lines_in_files(files);

    for (int i = 0; i < results.size(); i++) {
        std::cout << results[0] << "\n";
    }

    return 0;
}


Overwriting countlines01.cpp


In [ ]:
!g++ countlines01.cpp -o countlines01; ./countlines01

29


In [ ]:
!wc countlines01.cpp

 29  75 702 countlines01.cpp


行数が一致することを確認。

In [ ]:
# std::count アルゴリズムを使う
%%writefile countlines02.cpp
#include <algorithm>
#include <fstream>
#include <iostream>
#include <vector>

int count_lines(const std::string& filename) {
    std::ifstream in(filename);
    return std::count(std::istreambuf_iterator<char>(in),
                      std::istreambuf_iterator<char>(), '\n');
}
std::vector<int> count_lines_in_files(const std::vector<std::string>& files) {
    std::vector<int> results;
    for (const auto& file : files) {
        results.push_back(count_lines(file));
    }
    return results;
}

int main() {
    std::vector<std::string> files{"countlines01.cpp"};
    std::vector<int> results;

    results = count_lines_in_files(files);

    for (int i = 0; i < results.size(); i++) {
        std::cout << results[0] << "\n";
    }

    return 0;
}


Overwriting countlines02.cpp


In [ ]:
!g++ countlines02.cpp -o countlines02; ./countlines02

29


In [ ]:
%%writefile countlines03.cpp
#include <algorithm>
#include <fstream>
#include <iostream>
#include <vector>

int count_lines(const std::string& filename) {
    std::ifstream in(filename);
    return std::count(std::istreambuf_iterator<char>(in),
                      std::istreambuf_iterator<char>(), '\n');
}

std::vector<int> count_lines_in_files(const std::vector<std::string>& files) {
    std::vector<int> results(files.size());
    std::transform(files.cbegin(), files.cend(), results.begin(), count_lines);
    return results;
}

int main() {
    std::vector<std::string> files{"countlines01.cpp"};
    std::vector<int> results;

    results = count_lines_in_files(files);

    for (int i = 0; i < results.size(); i++) {
        std::cout << results[0] << "\n";
    }

    return 0;
}


Overwriting countlines03.cpp


In [ ]:
!g++ countlines03.cpp -o countlines03; ./countlines03

29


In [ ]:
# 次のコードはRangesを使っていて、いまの段階では動かない。
%%script false
%%writefile countlines04.cpp
#include <algorithm>
#include <fstream>
#include <iostream>
#include <vector>

int count_lines(const std::string& filename) {
    std::ifstream in(filename);
    return std::count(std::istreambuf_iterator<char>(in),
                      std::istreambuf_iterator<char>(), '\n');
}

std::vector<int> count_lines_in_files(const std::vector<std::string>& files) {
    return files | transform(count_lines);
}

int main() {
    std::vector<std::string> files{"countlines01.cpp"};
    std::vector<int> results;

    results = count_lines_in_files(files);

    for (int i = 0; i < results.size(); i++) {
        std::cout << results[0] << "\n";
    }

    return 0;
}


In [ ]:
# 次のコードはRangesを使っていて、いまの段階では動かない。
%%script false
%%writefile countlines05.cpp
#include <algorithm>
#include <fstream>
#include <iostream>
#include <vector>

int count_lines(const std::string& filename) {
    std::ifstream in(filename);
    return std::count(std::istreambuf_iterator<char>(in),
                      std::istreambuf_iterator<char>(), '\n');
}

std::vector<int> count_lines_in_files(const std::vector<std::string>& files) {
    return files | transform(open_file) | transform(count_lines);
}

int main() {
    std::vector<std::string> files{"countlines01.cpp"};
    std::vector<int> results;

    results = count_lines_in_files(files);

    for (int i = 0; i < results.size(); i++) {
        std::cout << results[0] << "\n";
    }

    return 0;
}


In [ ]:
!g++ countlines05.cpp -o countlines05; ./countlines05

In [ ]:
#実験
%%script false
%%writefile transform01.cpp
#include <algorithm>
#include <cctype>
#include <functional>
#include <iostream>
#include <string>
#include <vector>

int main() {
    std::string s("hello");

    namespace ranges = std::ranges;

    ranges::transform(
        s.begin(), s.end(), s.begin(),
        [](unsigned char c) -> unsigned char { return std::toupper(c); });

    std::vector<std::size_t> ordinals;
    ranges::transform(s, std::back_inserter(ordinals),
                      [](unsigned char c) -> std::size_t { return c; });

    std::cout << s << ':';
    for (auto ord : ordinals) {
        std::cout << ' ' << ord;
    }

    ranges::transform(ordinals, ordinals, ordinals.begin(), std::plus{});

    std::cout << '\n';
    for (auto ord : ordinals) {
        std::cout << ord << ' ';
    }
    std::cout << '\n';
}

Writing transform01.cpp


## この本のまとめ、感想
目次を見るとC++でモナドの話まで持っていこうとしている。 それはそれだが、わたしが目的としているのはそんな高度な話ではなくて、
1. for構文がイヤ。 やたらに出てくる。 これを vector で対応できないか。
1. c++20 の range を使いたいわけではない。 #include <arlgrithm> で使える transform() を使いたい。
ということは、vectorとtransformを勉強しよう。

# C++11でcopy_ifと無名関数が使えるようになったので改めて
という記事があって、これをなぞってみる。

## reduce() みたいな

In [7]:
%%writefile reduce01.cpp
#include <algorithm>
#include <iostream>
#include <vector>

std::vector<int> x = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10};
std::vector<int> y = {15, 12, 99, 27};
std::vector<int> z = {10, 20, 150, 100};
std::vector<int> w = {1, 3, 10, 100, -12, 2, 4};

int multiply(int x, int y) { return x * y; }

int gcd(int a, int b) {
    if (!b) return a;
    return gcd(b, a % b);
}

int lcd(int a, int b) { return a * b / gcd(a, b); }

int myMin(int a, int b) { return (a < b) ? a : b; }

int myMax(int a, int b) { return (a > b) ? a : b; }

int main() {
    std::cout << accumulate(x, x + x.size(), 0) << endl;
    std::cout << accumulate(x, x + x.size(), 1, multiply) << endl;
    std::cout << accumulate(y, y + y.size(), *y, gcd) << endl;
    std::cout << accumulate(z, z + z.size(), *z, lcd) << endl;
    std::cout << accumulate(w, w + w.size(), INF, myMin) << endl;
    std::cout << accumulate(w, w + w.size(), -INF, myMax) << endl;

    return 0;
}


Overwriting reduce01.cpp


In [8]:
!g++ reduce01.cpp -o reduce01; ./reduce01

reduce01.cpp: In function ‘int main()’:
reduce01.cpp:24:34: error: no match for ‘operator+’ (operand types are ‘std::vector<int>’ and ‘std::vector<int>::size_type {aka long unsigned int}’)
     std::cout << accumulate(x, x + x.size(), 0) << endl;
                                ~~^~~~~~~~~~
In file included from /usr/include/c++/7/vector:65:0,
                 from reduce01.cpp:3:
/usr/include/c++/7/bits/stl_bvector.h:387:3: note: candidate: std::_Bit_const_iterator std::operator+(std::ptrdiff_t, const std::_Bit_const_iterator&)
   operator+(ptrdiff_t __n, const _Bit_const_iterator& __x)
   ^~~~~~~~
/usr/include/c++/7/bits/stl_bvector.h:387:3: note:   no known conversion for argument 1 from ‘std::vector<int>’ to ‘std::ptrdiff_t {aka long int}’
/usr/include/c++/7/bits/stl_bvector.h:297:3: note: candidate: std::_Bit_iterator std::operator+(std::ptrdiff_t, const std::_Bit_iterator&)
   operator+(ptrdiff_t __n, const _Bit_iterator& __x)
   ^~~~~~~~
/usr/include/c++/7/bits/stl_bvector.h:297